In [1]:
import numpy as np
import pandas as pd

In [44]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
columns = ['NU_IDADE','TP_ANO_CONCLUIU','TP_ESCOLA','IN_TREINEIRO','TP_PRESENCA_CN',\
           'TP_PRESENCA_CH','TP_PRESENCA_LC','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO','Q006',\
           'Q026']
df_train = train[columns]
df_train['NU_NOTA_MT'] = train['NU_NOTA_MT']

/home/capizani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
df_train['Q026'] = pd.factorize(df_train['Q026'])[0]
df_train['Q006'] = pd.factorize(df_train['Q006'])[0]

/home/capizani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/capizani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 14 columns):
NU_IDADE           13730 non-null int64
TP_ANO_CONCLUIU    13730 non-null int64
TP_ESCOLA          13730 non-null int64
IN_TREINEIRO       13730 non-null int64
TP_PRESENCA_CN     13730 non-null int64
TP_PRESENCA_CH     13730 non-null int64
TP_PRESENCA_LC     13730 non-null int64
NU_NOTA_CN         10341 non-null float64
NU_NOTA_CH         10341 non-null float64
NU_NOTA_LC         10133 non-null float64
NU_NOTA_REDACAO    10133 non-null float64
Q006               13730 non-null int64
Q026               13730 non-null int64
NU_NOTA_MT         10133 non-null float64
dtypes: float64(5), int64(9)
memory usage: 1.5 MB


In [6]:
with_null_columns = ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO', 'NU_NOTA_MT']
for colum in with_null_columns:
    df_train[colum].fillna(0, inplace=True)

/home/capizani/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('NU_NOTA_MT',axis=1), 
                                                    df_train['NU_NOTA_MT'],test_size=0.3)

In [9]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

In [10]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
lin_pred = lin_reg.predict(X_test)

In [12]:
r2_score(y_test, lin_pred)

0.9125312890524703

In [13]:
rfr = RandomForestRegressor()
rfr

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [14]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Maximum number of levels in tree
max_depth = [None,3,5,7,9]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [15]:
grid_search = GridSearchCV(estimator = rfr, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [16]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 900 candidates, totalling 2700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/capizani/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 29.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 42.3min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 71.0min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 101.2min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 133.6min
[Parallel(n_jobs=-1)]: Done 2700 out of 2700 | elapsed: 141.8min finished


{'bootstrap': True,
 'max_depth': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 300}

In [20]:
params = grid_search.best_params_

In [21]:
params

{'bootstrap': True,
 'max_depth': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 300}

In [22]:
best_rfr = RandomForestRegressor(n_estimators=300, max_depth=7, min_samples_leaf=1,
                                min_samples_split=5, bootstrap=True)

In [23]:
best_rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [24]:
rfr_pred = best_rfr.predict(X_test)

In [25]:
r2_score(y_test, rfr_pred)

0.9244897572606084

In [26]:
rfr2 = RandomForestRegressor(max_depth=3, random_state=0)
abr = AdaBoostRegressor(random_state=0, n_estimators=100)

In [27]:
rfr2.fit(X_train, y_train)
abr.fit(X_train, y_train)

/home/capizani/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=100, random_state=0)

In [28]:
rfr2_pred = rfr2.predict(X_test)
abr_pred = abr.predict(X_test)

In [29]:
r2_score(y_test, rfr2_pred)

0.9197177626369467

In [30]:
r2_score(y_test, abr_pred)

0.8825962037731647

In [46]:
colunas = X_test.columns.tolist()
colunas.append('NU_INSCRICAO')
colunas

['NU_IDADE',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'IN_TREINEIRO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_REDACAO',
 'Q006',
 'Q026',
 'NU_INSCRICAO']

In [47]:
df_test = test[colunas]

In [53]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4576 entries, 0 to 4575
Data columns (total 14 columns):
NU_IDADE           4576 non-null int64
TP_ANO_CONCLUIU    4576 non-null int64
TP_ESCOLA          4576 non-null int64
IN_TREINEIRO       4576 non-null int64
TP_PRESENCA_CN     4576 non-null int64
TP_PRESENCA_CH     4576 non-null int64
TP_PRESENCA_LC     4576 non-null int64
NU_NOTA_CN         4576 non-null float64
NU_NOTA_CH         4576 non-null float64
NU_NOTA_LC         4576 non-null float64
NU_NOTA_REDACAO    4576 non-null float64
Q006               4576 non-null int64
Q026               4576 non-null int64
NU_INSCRICAO       4576 non-null object
dtypes: float64(4), int64(9), object(1)
memory usage: 500.6+ KB


In [51]:
df_test['Q026'] = pd.factorize(df_test['Q026'])[0]
df_test['Q006'] = pd.factorize(df_test['Q006'])[0]

/home/capizani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/capizani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
with_null_columns = ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO']
for colum in with_null_columns:
    df_test[colum].fillna(0, inplace=True)

/home/capizani/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [55]:
rfr_answer = best_rfr.predict(df_test.drop('NU_INSCRICAO', axis=1))

In [56]:
df_test['NU_NOTA_MT'] = rfr_answer

/home/capizani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
df_test[['NU_INSCRICAO','NU_NOTA_MT']].to_csv('answer.csv', index=False)

In [59]:
resposta = pd.read_csv('answer.csv')

In [60]:
resposta

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,430.414145
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,448.299965
2,b38a03232f43b11c9d0788abaf060f7366053b6d,592.284823
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.000000
4,715494628a50142ce8cb17191cfe6d0f3cae0934,546.593927
...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,452.061145
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,465.991943
4573,655fa6306720ff16e825903b5422a46608a77545,666.890433
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,445.153753
